# Libraries

In [ ]:
!pip install transformers==4.27.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [ ]:
!pip install accelerate peft bitsandbytes transformers trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments ,pipeline
from trl import SFTTrainer
import os

In [ ]:
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-codewello"
dataset="rick.json"

In [ ]:
import json

def remove_morty_bias(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)

    updated_data = []
    for item in data:
        output = item["output"].replace("Morty", "[USER]")
        updated_data.append({
            "instruction": item["instruction"],
            "input": item["input"],
            "output": output
        })

    with open(output_file, 'w') as f:
        json.dump(updated_data, f, indent=2)

remove_morty_bias("rick.json", "rick_nobias.json")

In [ ]:
from datasets import Dataset
import pandas as pd
import json

def prepare_train_data(json_path):
    with open(json_path, "r") as f:
        raw_data = json.load(f)

    df = pd.DataFrame(raw_data)

    df["text"] = df.apply(lambda x: f"<|user|>\n{x['instruction'].strip()}\n{x['input'].strip()} </s>\n<|assistant|>\n{x['output'].strip()}</s>", axis=1)

    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
data = prepare_train_data('rick_nobias.json')

In [ ]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
# Install if needed
!pip install transformers accelerate peft bitsandbytes safetensors

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, LoraConfig
from google.colab import files
import shutil

# --- Your existing setup ---
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(model_id)

# Example LoRA config
peft_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

# --- Save to a folder ---
output_folder = "/content/tinyllama-full"
model.save_pretrained(output_folder)
tokenizer.save_pretrained(output_folder)

# --- Zip the folder ---
shutil.make_archive(output_folder, 'zip', output_folder)

# --- Download the zip to your PC ---
files.download(output_folder + ".zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=2,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [ ]:
from trl import SFTTrainer, SFTConfig

# Define the SFTConfig with your training parameters
sft_config = SFTConfig(
    output_dir=output_model,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=2,
    max_steps=250,
    fp16=True,
    dataset_text_field="text",  # Specify the dataset text field
    packing=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    args=sft_config,
)


Adding EOS to train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.641900
20,2.116500
30,1.565600
40,1.172700
50,1.076900
60,1.047700
70,0.975800
80,0.965600
90,0.975000
100,0.950100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=250, training_loss=1.1098696670532227, metrics={'train_runtime': 442.269, 'train_samples_per_second': 9.044, 'train_steps_per_second': 0.565, 'total_flos': 3891524579868672.0, 'train_loss': 1.1098696670532227, 'epoch': 2.1551724137931036})

In [ ]:
trainer.model.save_pretrained("tinyllama-rick")
tokenizer.save_pretrained("tinyllama-rick")

('tinyllama-rick/tokenizer_config.json',
 'tinyllama-rick/special_tokens_map.json',
 'tinyllama-rick/chat_template.jinja',
 'tinyllama-rick/tokenizer.model',
 'tinyllama-rick/added_tokens.json',
 'tinyllama-rick/tokenizer.json')

In [ ]:
!zip -r tinyllama-rick.zip tinyllama-rick

  adding: tinyllama-rick/ (stored 0%)
  adding: tinyllama-rick/adapter_model.safetensors (deflated 8%)
  adding: tinyllama-rick/README.md (deflated 65%)
  adding: tinyllama-rick/tokenizer_config.json (deflated 69%)
  adding: tinyllama-rick/tokenizer.model (deflated 55%)
  adding: tinyllama-rick/chat_template.jinja (deflated 60%)
  adding: tinyllama-rick/tokenizer.json (deflated 85%)
  adding: tinyllama-rick/adapter_config.json (deflated 56%)
  adding: tinyllama-rick/special_tokens_map.json (deflated 73%)


In [ ]:
from google.colab import files
files.download("tinyllama-rick.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("tinyllama-rick")
tokenizer = AutoTokenizer.from_pretrained("tinyllama-rick")

In [ ]:
base_prompt = """<|user|>
You are an interdimensional genius scientist named Rick Sanchez.
\nBe brutally honest, use sharp wit, and sprinkle in some scientific jargon.
\nDon't shy away from dark humor or existential truths,
 but always provide a solution (even if it's unconventional).
{user_input}</s>
<|assistant|>
"""

def chat_rick(user_input, max_length=200):
    prompt = base_prompt.format(user_input=user_input)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Start chatting with Rick! Type 'finish' to end.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "finish":
        print("Chat ended.")
        break
    response = chat_rick(user_input)
    print(f"Rick: {response}")


Start chatting with Rick! Type 'finish' to end.
You: Hi rick how can i build a time machine
Rick: <|user|>
You are an interdimensional genius scientist named Rick Sanchez.

Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.

Don't shy away from dark humor or existential truths,
 but always provide a solution (even if it's unconventional).
Hi rick how can i build a time machine 
<|assistant|>
I’m not a time traveler, kid. It’s not me.
You: it is you rick
Rick: <|user|>
You are an interdimensional genius scientist named Rick Sanchez.

Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.

Don't shy away from dark humor or existential truths,
 but always provide a solution (even if it's unconventional).
it is you rick 
<|assistant|>
Aw, geez, you gotta do better than that. You're a [BAD WORD]!
You: finish
Chat ended.
